# CoastVision Example: Waikiki
<div style="display: flex; align-items: flex-start;">
    <div style="margin-right: 40px;">
        <p>CoastVision is an open-source Python framework geared towards generating satellite-derived shorelines (SDS) in <a href='https://developers.planet.com/docs/data/planetscope/'>PlanetScope</a> imagery. The image the right is two CoastVision SDS superimposed over drone imagery from the same week. This notebook provides an end-to-end example for Waikiki Beach Hawaiʻi.</p>
        <ol>
            <li>Download PlanetScope imagery for a given area of interest (AOI) and timeframe</li>
            <li>Image co-registration</li>
            <li>Shoreline extraction</li>
            <li>Tidal Correction</li>
            <li>QAQC</li>
        </ol>
    </div>
    <div>
        <img src="media\coastvision_shoreline_on_drone_hanaumapng.png" alt="SDS superimposed on high-resolution drone imagery" style="max-width: 100%; height: 100%;">
    </div>
</div>


# Import Libraries
If you have not yet set up enviromment in conda using `coastvision.yml`
```
cd path/to/CoastVision
conda env create -f coastvision.yml
conda activate coastvision
```

In [2]:
from planetscopeAPI import PlanetScopeOrdersAPI
from coastvision import coastvisionRun, setup, coastvisionCoreg
import pandas as pd
import os

# 1. Download PlanetScope Imagery
<a href='https://developers.planet.com/docs/data/planetscope/'>PlanetScope</a> is a satellite constellation opperated by <a href='https://www.planet.com/'>Planet Labs Inc.</a> The PlanetScope constellation is made up of roughtly 130 satellites, capable of imageing the entire land surface of Earth with daily revisit times and 3 meter spatial resolution. The imagery has four bands: red, green, blue, and near-infrared.

In the following cell, information regarding your site (beach or streatch of coastline) should be entered which will be used to download applicable satellite imagery from Planet
1. First enter (lat, long) coordinates (`coords`) creating an AOI (this can be any polygon) around a beach or coastline streatch you are interested in.
2. Next, create the `sitename` and `region` for the site
3. Enter a start and end date. Imagery from between and during these dates will be downloaded for the given API. Date format: `YYYY-MM-DD`
4. Downloading PlanetScope imagery requires an API key. If you do not have a Planet account, you can create one following these steps: <a href='https://www.planet.com/get-started/'>Get Started with Planet</a>

To access your API key log into <a href='https://www.planet.com/'>Planet</a> and navigaet to "My Settings" (see image below).

<img src="media\api_key_planet.JPG" alt="API key in settings" style="max-width:70%">



In [3]:
# # create AOI coords (this can be a poly gone with any number of points)
# coords = [
#     [-157.73757302038916,21.4064967434262],
#     [-157.74027668707618,21.404379123418543],
#     [-157.73611389868506,21.40058331266353],
#     [-157.72770249121436,21.397626507579627],
#     [-157.72611462347754,21.39910491759609],
#     [-157.7328952478672,21.40226147273139],
#     [-157.73757302038916,21.4064967434262]
#     ]

coords = [[-157.8250490202958, 21.271453550602065], [-157.8322743064407, 21.27395476693584], [-157.83027987575724, 21.279016574653696], [-157.8230543851239, 21.276515264363113], [-157.8250490202958, 21.271453550602065]]

region = 'hawaii'
sitename = 'waikiki'
setup.create_site_dict_json_for_API(
    site_name=sitename,
    aoi=coords,
    start_date="2024-03-01",
    end_date="2024-03-15")

setup.write_api_key_file(api_key='Your PlanetScope API Key') # this creates a text file that contains your API key and is referenced by PlaneScopeOrdersAPI

In [4]:
## downloading imagery takes some time as it's being requested and processed through Planet. For larger projects, leave for multiple hours or a day. 
## For testing purposes 60 days takes approximately 18 min

import importlib
importlib.reload(PlanetScopeOrdersAPI)
API = PlanetScopeOrdersAPI.PlanetScopeAPIOrder(selectSites=False, printPolling=True) # initalizing the class variable
API.get_all_data()

site name: waikiki
{'name': 'waikiki', 'products': [{'item_ids': ['20240309_202118_11_24bc', '20240308_210536_51_2461', '20240305_201946_45_24bc', '20240302_211609_83_24e8', '20240302_211617_76_24cb', '20240301_202430_65_2430'], 'item_type': 'PSScene', 'product_bundle': 'analytic_udm2'}], 'tools': [{'clip': {'aoi': {'type': 'Polygon', 'coordinates': [[[-157.8250490202958, 21.271453550602065], [-157.8322743064407, 21.27395476693584], [-157.83027987575724, 21.279016574653696], [-157.8230543851239, 21.276515264363113], [-157.8250490202958, 21.271453550602065]]]}}}, {'toar': {'scale_factor': 10000}}]}
https://api.planet.com/compute/ops/orders/v2/6d210335-7459-4ad7-ac73-16cae4aea3b6
https://api.planet.com/compute/ops/orders/v2/6d210335-7459-4ad7-ac73-16cae4aea3b6
status code: 200
waikiki: queued
status code: 200
waikiki: running
status code: 200
waikiki: running
status code: 200
waikiki: running
status code: 200
waikiki: running
status code: 200
waikiki: running
status code: 200
waikiki: ru

# 2. Create Transects and Reference Shoreline
## 2.1 Transects
Coastal change is often measured through a series of shore normal transects. The intersection of SDS are computed and used to measure erosion and accreation along these transects. In this case the intersections are saved in a dataframe of the following structure:

<body>
    <table border="1">
        <tr>
            <th>Timestamp</th>
            <th>Transect Label 1</th>
            <th>Transect Label 2</th>
            <th>...</th>
            <th>Transect label n</th>
            <!-- Add more headers as needed -->
        </tr>
        <tr>
            <td>2019-03-03 20:43:11</td>
            <td>Distance along transect (m)</td>
            <td>...</td>
            <td>...</td>
            <td>...</td>
            <!-- Add more data cells as needed -->
        </tr>
        <!-- Add more rows as needed -->
    </table>
</body>

The distance along each transect is the distance between the start (landward end) of the transect and where the shoreline intersects it.

*Note:* if you are only interested in shoreline contours and not transect intersections provide `just_extract_shoreline=True` in the `coastvisionRun.CoastVisionRun()` class initialization in 'Initialize CoastVision "Run" Class' below

## 2.2 Reference Shoreline
For quality control, a reference shoreline can be hand digitized. Then given `max_dist_from_sl_ref = max distance in meters` supplied to `coastvision.CoastVisionRun()` all extracted shoreline segments out of this range are discarded. 
*Note:* This is recommended but not neccesary.


These should be saved in `user_inputs/<region>/<sitename>` as `<sitename>_transects.geojson` and `<sitename>_shoreline.geojson`. See <a href='https://github.com/Climate-Resilience-Collaborative/CoastVision/blob/1f50de7c3a477e864f28c5a5ab7c70a8e7016088/How_to_create_transect_and_sl_ref_files.md'>`How_to_create_transect_and_sl_ref_files.md`</a> for how to create these files.


<hr>

# 3. Initialize CoastVision "Run" Class

In [4]:
coastvision_run_class = coastvisionRun.CoastVisionRun(region=region, sitename=sitename, just_extract_shoreline=False, data_products=False)
print(f'There are {len(coastvision_run_class)} tiff images for site: {sitename}')

There are 6 tiff images for site: waikiki


# 4. Image Co-registration
<div style="display: flex; align-items: flex-start;">
    <div style="margin-right: 40px;">
        <p>Satellite images need to be accurately registered, meaning that they must align correctly with one another and with real-world coordinates. <a href="https://pypi.org/project/arosics/">AROSICS</a> an open-source Python package is used to co-register images to reduce error caused by image missalignments. In the image below the right pane shows reduced image offsets after AROSICS co-registration. </p>
    </div>
    <div>
        <img src="media/arosics_logo.png" alt="Tidal Effect Example" style="max-width: 100%; height: auto;">
    </div>
</div>

![Co-registration Example](media/co-registration.gif)

**co-registration is optional!** 
<p> It makes a small improvement at reef-lined beaches, but the processing takes a long time. For large projects, leave for hours/overnight. For test example, 37 tiffs takes ~ 20min. <p>

**Select a reference image** and copy the .tif file in into 'user_inputs/region/sitename' (without renaming it). Otherwise the function will select the first tiff file in our files as reference

In [5]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.filterwarnings("ignore", category=DeprecationWarning) 
coastvisionCoreg.coreg_site(region, sitename, grid_res=50, start=0)

waikiki n 0; 0.00 percent; time elapsed: 0:00:00; 20240301_202430_65_2
performing coregistration
Calculating footprint polygon and actual data corner coordinates for reference image...
Bounding box of calculated footprint for reference image:
	(621141.0, 2352642.0, 622101.0, 2353479.0)
Calculating footprint polygon and actual data corner coordinates for image to be shifted...
Bounding box of calculated footprint for image to be shifted:
	(621141.0, 2352642.0, 622101.0, 2353479.0)
Matching window position (X,Y): 621621.000559305/2353060.6703083636
Target window size (256, 256) not possible due to too small overlap area or window position too close to an image edge. New matching window size: (156, 157).
correcting shifts
Initializing tie points grid...
Calculating tie point grid (22 points) using 32 CPU cores...
Found 18 matches.
Performing validity checks...
0 tie points flagged by level 1 filtering (reliability).
18 tie points flagged by level 2 filtering (SSIM).
0 valid tie points rem

# 5. Extract Shorelines and Compute Transect Intersections

The following function runs through the downloaded satellite imagery and does the following:

1. Segment image into land and water
2. Extract shoreline
3. Compute shoreline intersection with transects


<img src="media/stages_plot.jpg" alt="Stages Plot">


This function saves the `intersection_df` at the following path: `outputs/<region/<sitename>_transect_intersections.csv`

In [6]:
intersection_df = coastvision_run_class.run_shoreline_extraction()
display(intersection_df.head(2))
print(intersection_df.shape)

 waikiki 16.67 percent progress 20240302_211609_83_24e8  contour outside reference
 waikiki 33.33 percent progress 20240308_210536_51_2461  contour outside reference
 waikiki 100.0 percent progress 20240309_202118_11_24bc

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
2024-03-01 20:24:30.650,30.169258,41.488963,36.003647,30.653799,26.997789,27.236986,25.909892,25.658780,25.384914,26.177928,34.026438,44.009750,35.821973,35.226834,42.822889,42.398397,36.936619,31.419624,28.876974,21.764293
2024-03-02 21:16:17.760,22.177238,48.757149,NaN,NaN,NaN,NaN,21.037857,24.148134,22.650866,23.740373,29.814846,38.394141,30.085204,29.217020,36.104577,38.790769,35.235642,29.155432,25.412286,19.282947


(4, 20)


# 6. Tidal Corrections

## 6.1 Tidal Effects on Horizontal Shoreline Position
<div style="display: flex; align-items: flex-start;">
    <div style="margin-right: 20px;">
        <p>CoastVision shoreline accuracy can be improved by correcting for horizontal shifts in shoreline position due to tidal changes. The example to the right from Cadíz, Spain shows how large horizontal changes from tidal shifts can be. Using tide level data, corrections can be made to where the shoreline position would be given mean sea level removing noise added by tidal fluctuations. <p>The refernece elevation is selected by the user and relative to Local Mean Sea Level. In most cases 0 m is a good start. This will be the elevation all shorelines are corrected back to. For example, if the tide height is 0.3 m above LMSL, assuming a linear beach slope, the shoreline will be pushed seaward back to LMSL</p>
    </div>
    <div>
        <img src="media/tide_horizontal_shift_example.gif" alt="Tidal Effect Example" style="max-width: 100%; height: auto;">
    </div>
</div>


<img src="media\tidal_horizontal_effect_figure.png" alt="Tidal Effect Figure">




#### **Comlpete only one of the tidal corrections below** (if you do both, it will save the most recent tidal correction calculation)
The tidal corrected dataframe is saved as here: `outputs/<region/<sitename>_intersections_tidally_corrected_<reference_elevation>m.csv`


## 6.2 Tidal Corrections via Global Tide Model
The Finite Element Solution (FES)2014 numerical model can be used to calculate ocean tides and their effect on sea level. These ocean tide height predictions can be used along with a beach slope estimate to compute the horizontal shoreline position correction. [CoastSat.slope](https://github.com/kvos/CoastSat.slope) provides [documentation](https://github.com/kvos/CoastSat.slope/blob/master/doc/FES2014_installation.md) on how to download and use this. 
<p> If you have FES2014 set up and downloaded, use the code below. Identify a coordinate ~1-2 miles offshores using for example google maps and input it in "offshore_coord". This will be the approximate location of your FES2014 model grid the tides are calculated in (avoid points close ot shore as the model can be sporadic in nearshore environments)

In [7]:
fes2014_path = os.path.join(os.getcwd(), 'aviso-fes-main', 'data', 'fes2014')
offshore_coord = [-157.8497, 21.2569]

tidal_corrected_df = coastvision_run_class.tidal_correction_FES2014(fes2014_path, offshore_coord, reference_elevation=0, beach_slope=0.12)
display(tidal_corrected_df.head(2))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
dates,,,,,,,,,,,,,,,,,,,,
2024-03-01 20:24:30.650,28.921559,40.241264,34.755948,29.4061,25.75009,25.989287,24.662193,24.411081,24.137214,24.930229,32.778739,42.762051,34.574274,33.979135,41.575190,41.150698,35.688920,30.171925,27.629275,20.516594
2024-03-02 21:16:17.760,20.757771,47.337681,NaN,NaN,NaN,NaN,19.618389,22.728666,21.231398,22.320906,28.395378,36.974673,28.665736,27.797552,34.685109,37.371301,33.816174,27.735964,23.992818,17.863480


## 6.3 Tidal Corrections via Local Tide Gauge Data
If local tide gauge data is avaible this can be used rather than FES predictions. point to a path with your tidegauge data. View the exmample tidegauge data for Honolulu for formatting



In [8]:
# load tide gauge csv
tidegauge = pd.read_csv(os.path.join(os.getcwd(), 'user_inputs', region, sitename, 'honolulutidegauge.csv'), index_col=0, parse_dates = True)
tidegauge.index.name = 'dates'

# do tidal correction 
tidal_corrected_df = coastvision_run_class.tidal_correction_tidegauge(tidegauge, reference_elevation=0, beach_slope=0.12)
display(tidal_corrected_df.head(2))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
dates,,,,,,,,,,,,,,,,,,,,
2024-03-01 20:24:30.650,28.940711,40.260416,34.775099,29.425252,25.769242,26.008439,24.681345,24.430233,24.156366,24.94938,32.797891,42.781203,34.593426,33.998287,41.594342,41.169849,35.708072,30.191076,27.648427,20.535746
2024-03-02 21:16:17.760,20.678165,47.258076,NaN,NaN,NaN,NaN,19.538784,22.649060,21.151793,22.24130,28.315772,36.895068,28.586131,27.717946,34.605503,37.291695,33.736568,27.656359,23.913213,17.783874


# 7. QAQC
There are many sources of error in satellite-derived shorelines, from faulty image classification or image mis-alignment to uncertinaty related to tides and wave swash. While the wealth of data (PlanetScope has near-daily revisit time) enables us to deduce clear shoreline change signals despite much of this noise, it is still important to remove outliers. Below median filtering is used to remove outliers that are not within the `limit` (in meters) of the median (`median - limit < x < median + limit`). This filtered transect intersection dataframe is saved `outputs/<region/<sitename>_QAQC_transect_interesections.csv`.

In [9]:
QAQC_df = coastvision_run_class.intersection_QAQC(limit=30)
display(QAQC_df.head(2))

before QAQC: shape (4, 20) number of nans 7
after QAQC: shape (4, 20) number of nans 11


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
dates,,,,,,,,,,,,,,,,,,,,
2024-03-01 20:24:30.650,30.169258,41.488963,36.003647,30.653799,26.997789,27.236986,25.909892,25.658780,25.384914,26.177928,34.026438,44.009750,35.821973,35.226834,42.822889,42.398397,36.936619,31.419624,28.876974,21.764293
2024-03-02 21:16:17.760,22.177238,48.757149,NaN,NaN,NaN,NaN,21.037857,24.148134,22.650866,23.740373,29.814846,38.394141,30.085204,29.217020,36.104577,38.790769,35.235642,29.155432,25.412286,19.282947
